In [ ]:
%%capture
!pip install gdown

In [ ]:
import gdown
id = '1LB53KldZ_hG_Kc2wEBmRjLH0Js80WGgr'
gdown.download(id=id, output='trainval.zip')

In [ ]:
%%capture
!unzip trainval.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn.functional as F
from skimage import io, color
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from torchmetrics import JaccardIndex
from tqdm import tqdm
import gc

In [ ]:
# Load images from folders A and B
folder_A = 'trainval/A'
folder_B = 'trainval/B'
folder_label = 'trainval/label'

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu") # Use GPU or CPU for training

In [7]:
print(device)

cuda


In [8]:
# Function to load images from directory
def load_images_from_folder(folder, is_gray = True):
    images = []
    for filename in os.listdir(folder):
        img = io.imread(os.path.join(folder,filename)).astype(np.uint8)
        if img is not None:
            if is_gray:
                images.append(color.rgb2gray(img))
            else:    
                images.append(img)
    return images


In [9]:
images_A = load_images_from_folder(folder_A, is_gray=False)

In [10]:
images_B = load_images_from_folder(folder_B, is_gray=False)

In [11]:
labels = load_images_from_folder(folder_label, is_gray = False)

In [12]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(0.5, 0.5, 0.5),
])

In [13]:
class ChangeDetectionDataset(Dataset):
    def __init__(self, images_A, images_B, labels):
        self.images_A = images_A
        self.images_B = images_B
        self.labels = labels
#         self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_A = torch.tensor(self.images_A[idx], dtype=torch.float32).permute(2, 0, 1) / 255
        image_B = torch.tensor(self.images_B[idx], dtype=torch.float32).permute(2, 0, 1) / 255
        label = torch.tensor(self.labels[idx], dtype=torch.float32) / 255

#         if self.transform:
#             image_A = self.transform(image_A)
#             image_B = self.transform(image_B)
#             label = self.transform(label)

        return image_A, image_B, label

In [14]:
class DoubleConv(nn.Module):
    
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()

        self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
        self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)


In [15]:
class UNet_Model(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet_Model, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128))
        self.down2 = (Down(128, 256))
        self.down3 = (Down(256, 512))
        self.down4 = (Down(512, 1024))
        self.up1 = (Up(1024, 512))
        self.up2 = (Up(512, 256))
        self.up3 = (Up(256, 128))
        self.up4 = (Up(128, 64))
        self.outc = (OutConv(64, n_classes))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x_A, x_B):
        x = torch.cat([x_A, x_B], dim=1)
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        x = self.sigmoid(x)
        return x

    def use_checkpointing(self):
        self.inc = torch.utils.checkpoint(self.inc)
        self.down1 = torch.utils.checkpoint(self.down1)
        self.down2 = torch.utils.checkpoint(self.down2)
        self.down3 = torch.utils.checkpoint(self.down3)
        self.down4 = torch.utils.checkpoint(self.down4)
        self.up1 = torch.utils.checkpoint(self.up1)
        self.up2 = torch.utils.checkpoint(self.up2)
        self.up3 = torch.utils.checkpoint(self.up3)
        self.up4 = torch.utils.checkpoint(self.up4)
        self.outc = torch.utils.checkpoint(self.outc)
        self.sigmoid = torch.utils.checkpoint(self.sigmoid)

In [16]:
def dice_loss(pred, target, smooth=1):
    pred = pred.contiguous()
    target = target.contiguous()
    intersection = (pred * target).sum(dim=0).sum(dim=0)
    loss = (1 - ((2. * intersection + smooth) / (pred.sum(dim=0).sum(dim=0) + target.sum(dim=0).sum(dim=0) + smooth)))
    loss = loss.mean()
    loss += F.binary_cross_entropy(pred,target) * 0.0
    return loss

In [17]:
def train(images_A, images_B, labels, num_epochs=10, batch_size=32, learning_rate=0.01):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = UNet_Model(n_channels=6, n_classes=1).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
#     scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)
    print('Loading Data ...')
    dataset = ChangeDetectionDataset(images_A, images_B, labels)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    criterion = torch.nn.BCELoss()
    jaccard = JaccardIndex(task='binary').to(device)
    print('Training ...')
    for epoch_num in range(num_epochs):
#         scheduler.step()
        for param_group in optimizer.param_groups:
            print("LR", param_group['lr'])
        total_loss = 0
        total_intersections = 0
        total_unions = 0
        # number_of_batches = 0
        for a, b, l in tqdm(train_loader):
            # number_of_batches += 1
            a, b, l = a.to(device), b.to(device), l.to(device)
            optimizer.zero_grad()
            outputs = model(a,b)
            outputs = outputs.reshape((-1,l.shape[1],l.shape[2]))
            outputs_binary = (outputs>0.5) * 1
            total_intersections += np.logical_and(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
            total_unions += np.logical_or(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
#             loss = dice_loss(outputs,l,0)
            loss = criterion(outputs, l)
            total_loss += loss
            loss.backward()
            optimizer.step()
            
        epoch_loss = total_loss / len(images_A)
        epoch_jaccard = total_intersections / total_unions
        print(
        f'Epochs: {epoch_num + 1} | Loss: {epoch_loss} | Jaccard Index : {epoch_jaccard} \n')
        
    print('Validation ...')
    model.eval()
    val_total_intersections = 0
    val_total_unions = 0
    with torch.no_grad():
        for a, b, l in tqdm(train_loader):
            a, b, l = a.to(device), b.to(device), l.to(device)
            outputs = model(a,b)
            outputs = outputs.reshape((-1,l.shape[1],l.shape[2]))
            outputs_binary = (outputs>0.5) * 1
            val_total_intersections += np.logical_and(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
            val_total_unions += np.logical_or(outputs_binary.cpu().numpy(), l.cpu().numpy()).sum()
        
    val_jaccard = val_total_intersections / val_total_unions
    print(f'Validation Jaccard Index : {val_jaccard}')
    return model

In [18]:
model = train(images_A, images_B, labels, num_epochs=10, batch_size=32, learning_rate=0.1)

Loading Data ...
Training ...
LR 0.1


100%|██████████| 122/122 [01:37<00:00,  1.26it/s]


Epochs: 1 | Loss: 0.0049923793412745 | Jaccard Index : 0.0031861663632013963 

LR 0.1


100%|██████████| 122/122 [01:35<00:00,  1.27it/s]


Epochs: 2 | Loss: 0.00402836874127388 | Jaccard Index : 0.05325130984100951 

LR 0.1


...

Epochs: 9 | Loss: 0.0028462023474276066 | Jaccard Index : 0.44666728433218356 

LR 0.1


100%|██████████| 122/122 [01:35<00:00,  1.27it/s]


Epochs: 10 | Loss: 0.002835735445842147 | Jaccard Index : 0.44237585382272465 

Validation ...


100%|██████████| 122/122 [00:36<00:00,  3.33it/s]


Validation Jaccard Index : 0.4116219509593746


In [19]:
# gc.collect()
# torch.cuda.empty_cache()